In [11]:
import cv2           
import numpy as np   
import os            
from matplotlib import pyplot as plt



import time          
import mediapipe as mp  #pour la detection de corps
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical  




# Initialiser MediaPipe Holistic, qui est une solution pour la détection du corps entier
mp_holistic = mp.solutions.holistic 

# Initialiser MediaPipe Drawing Utils, qui contient des utilitaires pour dessiner les annotations
mp_drawing = mp.solutions.drawing_utils




def mediapipe_detection(image, model):
    # Convertir l'image de BGR en RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    
    # Rendre l'image non modifiable (non writeable)
    image.flags.writeable = False                  # Image is no longer writeable
    
    # Faire la prédiction en utilisant le modèle MediaPipe
    results = model.process(image)                 # Make prediction
    
    # Rendre l'image modifiable à nouveau
    image.flags.writeable = True                   # Image is now writeable
    
    # Convertir l'image de RGB en BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION RGB 2 BGR
    
    # Retourner l'image et les résultats de la prédiction
    return image, results




def draw_landmarks(image, results):
    # Dessiner les connexions du visage
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    
    # Dessiner les connexions de la pose (corps)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    
    # Dessiner les connexions de la main gauche
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    
    # Dessiner les connexions de la main droite
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

    
# draw_landmarks -> Function does not return the image but rather applies the landmark visualizations to the current image in place    



def draw_styled_landmarks(image, results):
    # Dessiner les connexions du visage
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             # Spécifications pour colorer les points de repère (points) du visage
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             # Spécifications pour colorer les connexions (lignes) du visage
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Dessiner les connexions de la pose (corps)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             # Spécifications pour colorer les points de repère (points) du corps
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             # Spécifications pour colorer les connexions (lignes) du corps
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Dessiner les connexions de la main gauche
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             # Spécifications pour colorer les points de repère (points) de la main gauche
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             # Spécifications pour colorer les connexions (lignes) de la main gauche
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Dessiner les connexions de la main droite  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             # Spécifications pour colorer les points de repère (points) de la main droite
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             # Spécifications pour colorer les connexions (lignes) de la main droite
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

    
#we are just formatting or updating our draw landmark function(that's not complusory though, u can use only standard drwa_landmark func)    
    



import cv2
import mediapipe as mp

# Initialiser la capture vidéo à partir de la webcam
cap = cv2.VideoCapture(0)

# Définir le modèle MediaPipe Holistic avec des seuils de détection et de suivi
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Lire le flux vidéo
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        # Faire les détections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Dessiner les points de repère
        draw_styled_landmarks(image, results)  # Utiliser la fonction améliorée pour dessiner les points de repère avec des couleurs différentes
        
        # Afficher à l'écran
        cv2.imshow('OpenCV Feed', image)

        # Quitter proprement
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Libérer la capture vidéo et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()


# Compter le nombre de points de repère détectés sur la pose (corps)
num_pose_landmarks = len(results.pose_landmarks.landmark) 


results


# Appeler la fonction pour dessiner les points de repère sur le visage, le corps, et les mains
draw_landmarks(frame, results)




pose = []               
# Pour chaque point de repère de la pose détecté, crée un tableau numpy avec les coordonnées x, y, z et la visibilité, puis ajoute-le à la liste 'pose'.
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)




# Crée un tableau numpy plat avec les coordonnées x, y, z et la visibilité des points de repère de la pose; sinon, initialise un tableau de zéros de taille 132 si aucun point de repère n'est détecté.
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

# Crée un tableau numpy plat avec les coordonnées x, y, z des points de repère du visage; sinon, initialise un tableau de zéros de taille 1404 si aucun point de repère n'est détecté.
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

# Crée un tableau numpy plat avec les coordonnées x, y, z des points de repère de la main gauche; sinon, initialise un tableau de zéros de taille 63 si aucun point de repère n'est détecté.
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

# Crée un tableau numpy plat avec les coordonnées x, y, z des points de repère de la main droite; sinon, initialise un tableau de zéros de taille 63 si aucun point de repère n'est détecté.
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)



def extract_keypoints(results):
    # Crée un tableau numpy plat avec les coordonnées x, y, z et la visibilité des points de repère de la pose; sinon, initialise un tableau de zéros de taille 132 si aucun point de repère n'est détecté.
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    # Crée un tableau numpy plat avec les coordonnées x, y, z des points de repère du visage; sinon, initialise un tableau de zéros de taille 1404 si aucun point de repère n'est détecté.
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    # Crée un tableau numpy plat avec les coordonnées x, y, z des points de repère de la main gauche; sinon, initialise un tableau de zéros de taille 63 si aucun point de repère n'est détecté.
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    # Crée un tableau numpy plat avec les coordonnées x, y, z des points de repère de la main droite; sinon, initialise un tableau de zéros de taille 63 si aucun point de repère n'est détecté.
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    # Concatène tous les tableaux de points de repère (pose, visage, main gauche, main droite) en un seul tableau numpy.
    return np.concatenate([pose, face, lh, rh])



result_test = extract_keypoints(results)



np.save('0', result_test)   
# en enregistrant chaque image sous forme de tableau numpy à l'intérieur de notre dossier 'MP_Data', nous aurons donc 30 tableaux numpy dans chaque dossier d'actions


np.load('0.npy')

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect, including Tunisian Sign Language actions in French (transliterated)
actions = np.array([ 'n3ass','lunettes','je froid','3aslama', 'Chokran', 'Nhebbek', 'La',  'j aime', 'j aime pas', 'Nakel', 'Nchrob' ,  'Bahi',
      'Telephone',   's il te plait', 'pleur','fort', 'lissar','Waqtach',
    'Limine' , 'sghir', 'kbir', 'stop', 'sma3ni','fakar','chof'])





#Trente vidéos de données
no_sequences = 30

# Les vidéos dureront 30 images
sequence_length = 30


# Vérifie si le chemin de répertoire spécifié par DATA_PATH n'existe pas.
# Si le répertoire n'existe pas, il le crée.
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)



# Itère sur chaque action dans la liste d'actions
for action in actions: 
    # Pour chaque action, itère sur le nombre de séquences
    for sequence in range(no_sequences):
        try:
            # Essaye de créer un répertoire pour l'action et la séquence spécifiée dans DATA_PATH
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            # Ignore les erreurs (par exemple, si le répertoire existe déjà) et continue
            pass



label_map = {label:num for num, label in enumerate(actions)}




# Initialise deux listes vides pour stocker les séquences et les étiquettes
sequences, labels = [], []

# Itère à travers chaque action dans la liste d'actions
for action in actions:
    # Pour chaque action, itère à travers chaque séquence (vidéo)
    for sequence in range(no_sequences):
        # Crée une liste pour stocker les frames de la séquence actuelle
        window = []
        # Itère à travers chaque frame de la séquence
        for frame_num in range(sequence_length):
            # Charge les points de repère extraits (au format .npy) pour la frame courante
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # Ajoute les points de repère à la fenêtre (séquence)
            window.append(res)
        # Ajoute la fenêtre (séquence complète) à la liste des séquences
        sequences.append(window)
        # Ajoute l'étiquette correspondant à l'action à la liste des étiquettes
        labels.append(label_map[action])


X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard



# Définit le chemin du répertoire où les journaux TensorBoard seront sauvegardés
log_dir = os.path.join('Logs')

# Crée une instance de TensorBoard pour enregistrer les journaux d'entraînement dans le répertoire spécifié
tb_callback = TensorBoard(log_dir=log_dir)



# Crée un modèle séquentiel
model = Sequential()

# Ajoute une couche LSTM avec 64 unités, renvoie les séquences (True) pour la connexion avec la couche suivante,
# utilise la fonction d'activation ReLU et spécifie la forme d'entrée de (40, 1662)
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(40,1662)))

# Ajoute une deuxième couche LSTM avec 128 unités, renvoie les séquences (True) pour la connexion avec la couche suivante,
# utilise la fonction d'activation ReLU
model.add(LSTM(128, return_sequences=True, activation='relu'))

# Ajoute une troisième couche LSTM avec 64 unités, ne renvoie pas les séquences (False),
# utilise la fonction d'activation ReLU
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Ajoute une couche Dense avec 64 unités et la fonction d'activation ReLU
model.add(Dense(64, activation='relu'))

# Ajoute une couche Dense avec 32 unités et la fonction d'activation ReLU
model.add(Dense(32, activation='relu'))

# Ajoute une couche Dense finale avec un nombre d'unités égal au nombre d'actions,
# avec la fonction d'activation softmax pour la classification multi-classe
model.add(Dense(actions.shape[0], activation='softmax'))



# 'res' contient les probabilités de prédiction pour trois classes.
# Les valeurs représentent la probabilité que l'entrée appartienne à chaque classe respective.
# Ici, 0.7 indique la probabilité la plus élevée pour la première classe,
# suivie de 0.2 pour la deuxième classe, et 0.1 pour la troisième classe.
res = [.7, 0.2, 0.1]


# Compile le modèle en spécifiant l'optimiseur, la fonction de perte et les métriques à utiliser pendant l'entraînement
model.compile(
    optimizer='Adam',                # Utilise l'optimiseur Adam, qui est un algorithme d'optimisation adaptatif couramment utilisé pour la formation des réseaux de neurones
    loss='categorical_crossentropy', # Spécifie la fonction de perte comme l'entropie croisée catégorique, appropriée pour les problèmes de classification multi-classe où les étiquettes sont encodées en one-hot
    metrics=['categorical_accuracy']  # Utilise la précision catégorique comme métrique d'évaluation pour suivre la performance du modèle pendant l'entraînement et l'évaluation
)


model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])


model.summary()

res = model.predict(X_test)

model.save('action.h5')
# Charge les poids préalablement sauvegardés dans le fichier 'action.h5' dans le modèle
model.load_weights('action.h5')

c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - categorical_accuracy: 0.0342 - loss: 3.8584
Epoch 2/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 0.0408 - loss: 3.2757
Epoch 3/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - categorical_accuracy: 0.0699 - loss: 3.2977
Epoch 4/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 0.1071 - loss: 3.1587
Epoch 5/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - categorical_accuracy: 0.0805 - loss: 3.1694
Epoch 6/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - categorical_accuracy: 0.0946 - loss: 3.2626
Epoch 7/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 0.0841 - loss: 3.3403
Epoch 8/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 0.0920 - loss: 3.9779
Epoch 9/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 0.0481 - loss: 3.1853
Epoch 10/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - categorical_accuracy: 0.0672 - loss: 3.1824
Epoch 11/200
23/23

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_15 (LSTM)                  │ (None, 40, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_16 (LSTM)                  │ (None, 40, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 25)             │           825 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,792,205 (6.84 MB)

 Trainable params: 597,401 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,194,804 (4.56 MB)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step  


In [20]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
yhat = model.predict(X_test)


# convertit les prédictions de leur représentation codée à chaud en une étiquette catégorielle, par exemple 0,1,2 sous la forme [1,0,0],[0,1,0],[0,0,1]
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)

accuracy_score(ytrue, yhat)


# Calculer les métriques
accuracy = accuracy_score(ytrue, yhat)
precision = precision_score(ytrue, yhat, average='weighted')
recall = recall_score(ytrue, yhat, average='weighted')
f1 = f1_score(ytrue, yhat, average='weighted')
conf_matrix = confusion_matrix(ytrue, yhat)



# Afficher les métriques
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall (Sensitivity):", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_report(ytrue, yhat))



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Accuracy: 0.05263157894736842
Precision: 0.05263157894736842
Recall (Sensitivity): 0.05263157894736842
F1 Score: 0.05263157894736842
Confusion Matrix:
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 

c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

In [21]:
colors = [(245,117,16), (117,245,16), (16,117,245)]* (len(actions) // 3 + 1)
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame
# sequence.reverse()

sequences = np.array(sequences, dtype=np.float32)  # Assurez-vous que toutes les séquences sont en float32
sequences = sequences[::-1]  # Reverses the entire sequences array

print(sequences)


sequences = np.array(sequences, dtype=np.float32)  # Ensure all sequences are in float32
for seq in sequences:
    print(len(seq))


# sequence.append('def')

# Append 'def' to the numpy array
sequence = np.append(sequence, 'def')
print(sequence)


#sequence.reverse()
sequences = sequences[::-1] 


sequence[-40:]


# 1. Variables pour la détection des gestes
sequence = []            # Liste pour stocker les séquences de points clés
sentence = []            # Liste pour stocker les actions détectées successivement
predictions = []        # Liste pour stocker les prédictions du modèle
threshold = 0.8          # Seuil de confiance pour valider une prédiction

# Ouvrir la capture vidéo à partir de la webcam
cap = cv2.VideoCapture(0)

# Initialiser le modèle Mediapipe pour la détection des gestes
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Lire une image du flux vidéo
        ret, frame = cap.read()

        # Effectuer des détections sur l'image
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Dessiner les repères sur l'image
        draw_styled_landmarks(image, results)
        
        # 2. Logique de prédiction
        keypoints = extract_keypoints(results)  # Extraire les points clés de l'image
        sequence.append(keypoints)             # Ajouter les points clés à la séquence
        sequence = sequence[-20:]              # Conserver seulement les 20 derniers frames
        
        if len(sequence) == 20:                # Si la séquence contient 20 frames
            res = model.predict(np.expand_dims(sequence, axis=0))[0]  # Prédire l'action à partir de la séquence
            print(actions[np.argmax(res)])    # Afficher l'action prédite
            predictions.append(np.argmax(res))  # Ajouter la prédiction à la liste des prédictions
            
            # 3. Logique de visualisation
            if np.unique(predictions[-10:])[0] == np.argmax(res):  # Si la prédiction dominante sur les 10 derniers frames est constante
                if res[np.argmax(res)] > threshold:  # Vérifier si la probabilité est au-dessus du seuil
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:  # Ajouter l'action à la sentence si elle est différente de la précédente
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])  # Ajouter la première action détectée à la sentence

            if len(sentence) > 5:  # Limiter la taille de la sentence à 5 actions
                sentence = sentence[-5:]

            # Visualiser les probabilités de chaque action
            image = prob_viz(res, actions, image, colors)
            
        # Dessiner un fond coloré et ajouter le texte de la sentence sur l'image
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Afficher l'image avec les visualisations
        cv2.imshow('OpenCV Feed', image)

        # Quitter la boucle lorsque la touche 'q' est pressée
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Libérer la capture vidéo et fermer toutes les fenêtres ouvertes
cap.release()
cv2.destroyAllWindows()


cap.release()
cv2.destroyAllWindows()

res[np.argmax(res)] > threshold

model.predict(np.expand_dims(X_test[0], axis=0))



[[[ 0.5160182   0.4524754  -1.0802364  ...  0.45198488  0.69363344
   -0.00785414]
  [ 0.5160037   0.4525295  -1.067734   ...  0.45351848  0.6975571
   -0.01019249]
  [ 0.5159935   0.45164716 -1.1196542  ...  0.45211238  0.69272876
   -0.00923952]
  ...
  [ 0.51605225  0.4523972  -1.1294849  ...  0.4531132   0.69589317
   -0.00204419]
  [ 0.5160483   0.452478   -1.1026971  ...  0.4511778   0.7013948
   -0.00705594]
  [ 0.516053    0.45250475 -1.1198027  ...  0.45517576  0.700079
   -0.00563149]]

 [[ 0.51374215  0.4563693  -1.1570764  ...  0.4525665   0.69688743
   -0.00573525]
  [ 0.5136515   0.4536169  -1.1316112  ...  0.44940856  0.68914217
   -0.01146954]
  [ 0.51458037  0.45280463 -1.1231663  ...  0.45195127  0.6892089
   -0.0045726 ]
  ...
  [ 0.5159159   0.45236513 -1.1198636  ...  0.44984734  0.694088
   -0.01629725]
  [ 0.5159333   0.45235026 -1.1080568  ...  0.45124978  0.69292927
   -0.00805763]
  [ 0.51588714  0.45240363 -1.1216     ...  0.45065275  0.6950686
   -0.00725913

c:\Users\Nessrine\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

array([[0.04322484, 0.04232845, 0.04294547, 0.0400345 , 0.04105075,
        0.04130815, 0.04245844, 0.03879448, 0.04043518, 0.04310024,
        0.04106274, 0.03941403, 0.04042232, 0.03979805, 0.00578663,
        0.04049407, 0.04022788, 0.04403084, 0.04240776, 0.03878455,
        0.0412681 , 0.04107919, 0.04194118, 0.04368616, 0.04391601]],
      dtype=float32)